In [ ]:
import os, sys
sys.path.append('/root/catkin_ws/src/primitives/')
import pickle
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import trimesh
import networkx

from open3d import JVisualizer

import copy
import time
import argparse
import numpy as np
from multiprocessing import Process, Pipe, Queue
import pickle
import rospy
import copy
import signal
import open3d
from IPython import embed

from yacs.config import CfgNode as CN
from closed_loop_experiments import get_cfg_defaults

from airobot import Robot
from airobot.utils import pb_util
from airobot.sensor.camera.rgbdcam_pybullet import RGBDCameraPybullet
from airobot.utils import common
import pybullet as p

from helper import util
from macro_actions import ClosedLoopMacroActions, YumiGelslimPybulet
# from closed_loop_eval import SingleArmPrimitives, DualArmPrimitives

In [ ]:
sys.path.append('/root/training/')

import os
import argparse
import time
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from data_loader import DataLoader
from model import VAE, GoalVAE
from util import to_var, save_state, load_net_state, load_seed, load_opt_state

import scipy.signal as signal

In [ ]:
# with open ('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/2.pkl', 'rb') as f:
#     grasp_data = pickle.load(f)
    
# print(grasp_data.keys())

In [ ]:
# with open ('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0_fixed/1003.pkl', 'rb') as f:
#     pull_data = pickle.load(f)
def get_rand_data(primitive='pull'):
    got_file = False
    while not got_file:
        val = np.random.randint(low=0, high=5000)
        if primitive == 'pull':
            test_path = '/root/catkin_ws/src/primitives/data/pull/pull_face_all_0/test/%d.pkl' % val
        elif primitive == 'grasp':
            test_path = '/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0_fixed/test/%d.pkl' % val
        if os.path.exists(test_path):
            with open (test_path, 'rb') as f:
    #             pull_data = pickle.load(f)
                data = pickle.load(f)
                got_file = True

    print(test_path)
    # print(pull_data.keys())
    print(data.keys())
    return data
pull_data = get_rand_data(primitive='pull')
grasp_data = get_rand_data(primitive='grasp')

# Training data: 
## Inputs: 
Initial observation, different representations:
- ```'start'```: pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```
- ```'keypoints_start'```: 3D location of box corners at start pose
- ```'obs':'pcd_pts'```: Point cloud of box from 3 different viewpoints, with same global coordinate sys. Can be fused with np.concatenate (see below)

Goal, different representations:
- ```'goal'```: pose
- ```'keypoints_goal'```: 3D location of box corners at goal pose

In [ ]:
# start observation
start = pull_data['start']
keypoints_start = pull_data['keypoints_start']
pcd_pts = pull_data['obs']['pcd_pts']
pcd_pts_start = np.concatenate(pcd_pts, axis=0)

# goal
goal = pull_data['goal']
keypoints_goal = pull_data['keypoints_goal_corrected']

# # start observation
# start = grasp_data['start']
# keypoints_start = grasp_data['keypoints_start']
# pcd_pts = grasp_data['obs']['pcd_pts']
# pcd_pts_start = np.concatenate(pcd_pts, axis=0)

# # goal
# goal = grasp_data['goal']
# keypoints_goal = grasp_data['keypoints_goal']

# Outputs:
## Pulling/Pushing (single arm)
Robot palm pose in the object frame, for active arm -- active arm currently based on which side of the table the object starts on. TODO perhaps includes predicting which arm is active, once we move to more diverse data. For now, everything for pulling happens with the right arm
- ```'contact_obj_frame'```: pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose

## Grasping/Pivoting (dual arm)

Right and left robot palm pose in the object frame
- ```'contact_obj_frame':'right'```: right palm pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose
- ```'contact_obj_frame':'left'```: left palm pose ```[x_pos, y_pos, z_pos, x_ori, y_ori, z_ori, w_ori]```, specified with respect to the coordinate system located at the object center of mass at the start pose

In [ ]:
# right palm contact for pulling
# contact_r = pull_data['contact_obj_frame']['right']

# both palms contact for grasping
contact_r = grasp_data['contact_obj_frame']['right']
contact_l = grasp_data['contact_obj_frame']['left']

In [ ]:
def pose_to_list(palm_pose_tensor):
    pos = palm_pose_tensor[:3].data.cpu().numpy()
    ori = palm_pose_tensor[3:].data.cpu().numpy()
    
    ori = ori/np.linalg.norm(ori)
    
    return pos.tolist() + ori.tolist()

# Testing

In [ ]:
# load trained model

# vae = VAE(14, 7, 2, 0.0003)
# load_net_state(vae, '/root/training/saved_models/pose_init_small_batch_1/pose_init_small_batch_1_epoch_63.pt')

# vae = VAE(31, 7, 3, 0.0003)
# load_net_state(vae, '/root/training/saved_models/pull_keypoints_init_small_batch_0/pull_keypoints_init_small_batch_0_epoch_99.pt')

# vae = VAE(14, 14, 3, 0.0003)
# load_net_state(vae, '/root/training/saved_models/grasp_pose_init_small_batch_0/grasp_pose_init_small_batch_0_epoch_298.pt')

# vae = VAE(31, 14, 3, 0.0003)
# load_net_state(vae, '/root/training/saved_models/grasp_keypoints_init_small_batch_1/grasp_keypoints_init_small_batch_1_epoch_199.pt')

# vae = VAE(31, 7, 3, 0.0003)
# print(vae.encoder)
# print(vae.decoder)
# load_net_state(vae, '/root/training/saved_models/grasp_keypoints_two_heads_small_batch_r_only_0/grasp_keypoints_two_heads_small_batch_r_only_0_epoch_5.pt')

# vae = VAE(31+14, 7, 3, 31, 0.0003)
# print(vae.encoder)
# print(vae.decoder)
# load_net_state(vae, '/root/training/saved_models/grasp_keypoints_two_heads_full_data_grasp_input_1/grasp_keypoints_two_heads_full_data_grasp_input_1_epoch_199.pt')

# vae = VAE(24+7+7, 7, 3, 24+7, 0.0003)
# print(vae.encoder)
# print(vae.decoder)
# load_net_state(vae, '/root/training/saved_models/pull_keypoints_two_heads_diverse_goals_full_data_palm_input_0/pull_keypoints_two_heads_diverse_goals_full_data_palm_input_0_epoch_199.pt')


# vae = GoalVAE(24+7, 7, 2, 24, 0.0003)
# print(vae.encoder)
# print(vae.decoder)
# load_net_state(vae, '/root/training/saved_models/pull_keypoints_goal_small_batch_0/pull_keypoints_goal_small_batch_0_epoch_137.pt')

vae = GoalVAE(7, 7, 4, 0, 0.0003)
print(vae.encoder)
print(vae.decoder)
load_net_state(vae, '/root/training/saved_models/pull_transformation_small_batch_lr_1e2_ratio_test_1/pull_transformation_small_batch_lr_1e2_ratio_test_1_epoch_16.pt')


# vae = GoalVAE(7, 7, 4, 0, 0.0003)
# print(vae.encoder)
# print(vae.decoder)
# load_net_state(vae, '/root/training/saved_models/grasp_transformation_small_batch_lr_1e2_ratio_test_0/grasp_transformation_small_batch_lr_1e2_ratio_test_0_epoch_199.pt')

In [ ]:
# sample from the latent space and use the decoder/generator to produce contacts, visualize below
# x = torch.from_numpy(np.asarray(start+goal, dtype=np.float32))
# x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))

x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))

# contact_input = torch.from_numpy(np.asarray(contact_r+contact_l, dtype=np.float32))
contact_input = torch.from_numpy(np.asarray(contact_r, dtype=np.float32))

decoder_input = x[:24+7]
z, pose_sample, z_mu, z_logvar = output = vae.forward(torch.cat((x, contact_input), dim=0), decoder_input)

In [ ]:
print("mu: " + str(z_mu) + ", std: " + str(torch.exp(0.5*z_logvar)))

In [ ]:
# pose = vae.decode(z)
# pose = vae.decode(z_mu)

z_sample = torch.randn_like(torch.zeros(3, ))

pose = vae.decode(z_sample, decoder_input)
print(pose)

In [ ]:
# pose_r = pose[:7]
# pose_l = pose[7:]

pose_r = pose[0]
pose_l = pose[1]

In [ ]:
pull_data_eval = copy.deepcopy(pull_data)
pull_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)

# grasp_data_eval = copy.deepcopy(grasp_data)
# grasp_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# grasp_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

In [ ]:
print("data: " + str(pull_data['contact_obj_frame']['right']))
print("--------")
print("eval: " + str(pull_data_eval['contact_obj_frame']['right']))

# print("right: " + str(grasp_data_eval['contact_obj_frame']['right']))
# print("----------------------------")
# print("left: " + str(grasp_data_eval['contact_obj_frame']['left']))

# Visualization

In [ ]:
with open('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0/metadata.pkl', 'rb') as mf:
    metadata = pickle.load(mf)
# with open('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/metadata.pkl', 'rb') as mf:
#     metadata = pickle.load(mf)


print('Metadata keys: ')
dynamics_info = metadata['dynamics']
mesh_file = metadata['mesh_file']
palm_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/mpalm/mpalms_all_coarse.stl'
table_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/table/table_top.stl'
cfg = metadata['cfg']

# Visualize contact on object

In [ ]:
def vis_palms(data, name='pull', goal='transformation'):
    obj_mesh = trimesh.load_mesh(mesh_file)
    r_palm_mesh = trimesh.load_mesh(palm_mesh_file)
    l_palm_mesh = trimesh.load_mesh(palm_mesh_file)
    table_mesh = trimesh.load_mesh(table_mesh_file)
    
    obj_pos_world = data['start'][:3]
    obj_ori_world = data['start'][3:]
    obj_ori_mat = common.quat2rot(obj_ori_world)
    h_trans = np.zeros((4, 4))
    h_trans[:3, :3] = obj_ori_mat
    h_trans[:-1, -1] = obj_pos_world
    h_trans[-1, -1] = 1

    obj_mesh.apply_transform(h_trans)
    
    goal_obj_mesh = trimesh.load_mesh(mesh_file)
    
    if goal == 'transformation':
        T_mat = util.matrix_from_pose(util.list2pose_stamped(data['transformation_corrected']))
        T_start = util.matrix_from_pose(util.list2pose_stamped(data['start']))
        T_goal = np.matmul(T_mat, T_start)
        goal_obj_pose_world = util.pose_stamped2list(util.pose_from_matrix(T_goal))
#         goal_obj_pose_start = util.list2pose_stamped(data['transformation_corrected'])
#         goal_obj_pose_world = util.pose_stamped2list(
#             util.convert_reference_frame(
#                 goal_obj_pose_start,
#                 util.unit_pose(),
#                 util.list2pose_stamped(data['start'])))
        goal_obj_pos_world = goal_obj_pose_world[:3]
        goal_obj_ori_world = goal_obj_pose_world[3:]
    else:
        goal_obj_pos_world = data['goal'][:3]
        goal_obj_ori_world = data['goal'][3:]
    goal_obj_ori_mat = common.quat2rot(goal_obj_ori_world)
    goal_h_trans = np.zeros((4, 4))
    goal_h_trans[:3, :3] = goal_obj_ori_mat
    goal_h_trans[:-1, -1] = goal_obj_pos_world
    goal_h_trans[-1, -1] = 1 
    
    goal_obj_mesh.apply_transform(goal_h_trans)
    
    if name == 'pull':
        tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame']['right'])
        tip_contact_r = util.convert_reference_frame(
            pose_source=tip_contact_r_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))

        wrist_contact_r = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_r)

        wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
        
        palm_pos_world_r = wrist_contact_r_list[:3]
        palm_ori_world_r = wrist_contact_r_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_r)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_r
        h_trans[-1, -1] = 1

        r_palm_mesh.apply_transform(h_trans)      
        
        scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh, goal_obj_mesh])        
    else:
        tip_contact_r_obj = util.list2pose_stamped(data['contact_obj_frame']['right'])
        tip_contact_l_obj = util.list2pose_stamped(data['contact_obj_frame']['left'])

        tip_contact_r = util.convert_reference_frame(
            pose_source=tip_contact_r_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))
            
        tip_contact_l = util.convert_reference_frame(
            pose_source=tip_contact_l_obj,
            pose_frame_target=util.unit_pose(),
            pose_frame_source=util.list2pose_stamped(data['start']))            
            
        wrist_contact_r = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_r)

        wrist_contact_l = util.convert_reference_frame(
            pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
            pose_frame_target=util.unit_pose(),
            pose_frame_source=tip_contact_l)

        wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
        wrist_contact_l_list = util.pose_stamped2list(wrist_contact_l)
        
        palm_pos_world_r = wrist_contact_r_list[:3]
        palm_ori_world_r = wrist_contact_r_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_r)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_r
        h_trans[-1, -1] = 1

        r_palm_mesh.apply_transform(h_trans)
        
        palm_pos_world_l = wrist_contact_l_list[:3]
        palm_ori_world_l = wrist_contact_l_list[3:]
        palm_ori_mat = common.quat2rot(palm_ori_world_l)
        h_trans = np.zeros((4, 4))
        h_trans[:3, :3] = palm_ori_mat
        h_trans[:-1, -1] = palm_pos_world_l
        h_trans[-1, -1] = 1

        l_palm_mesh.apply_transform(h_trans)        
        

        scene = trimesh.Scene([obj_mesh, r_palm_mesh, l_palm_mesh, table_mesh, goal_obj_mesh])
#         scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh])
    good_camera_euler = [1.0513555,  -0.02236318, -1.62958927]
    box_count = 0
    for key in scene.geometry.keys():
        print(key)
        if 'mpalms_all_coarse' in key:
            scene.geometry[key].visual.face_colors = [100, 100, 0, 30]
        if 'realsense_box_experiments' in key:
            if box_count == 0:
                scene.geometry[key].visual.face_colors = [250, 200, 200, 150]
                box_count += 1
            else:
                scene.geometry[key].visual.face_colors = [200, 200, 250, 150]                

    scene.geometry['table_top.stl'].visual.face_colors = [200, 200, 200, 250]
#     scene.geometry['realsense_box_experiments.stl'].visual.face_colors = [200, 200, 250, 250]
    scene.set_camera(angles=good_camera_euler, center=data['start'][:3], distance=0.8)
    return scene

In [ ]:
def correct_pos(data):
    contact_obj_frame_pred = util.list2pose_stamped(pull_data_eval['contact_obj_frame'])
    contact_world_frame = util.convert_reference_frame(contact_obj_frame_pred, util.unit_pose(), util.list2pose_stamped(start))
    contact_pos = open3d.utility.DoubleVector(np.array(util.pose_stamped2list(contact_world_frame)[:3]))

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_pts']))
    pcd.colors = open3d.utility.Vector3dVector(np.concatenate(pull_data_eval['obs']['pcd_colors']) / 255.0)

    kdtree = open3d.geometry.KDTreeFlann(pcd)
    nearest_pt_ind = kdtree.search_knn_vector_3d(contact_pos, 1)[1][0]
    nearest_pt_world = np.asarray(pcd.points)[nearest_pt_ind]

    contact_world_frame_corrected = copy.deepcopy(contact_world_frame)
    contact_world_frame_corrected.pose.position.x = nearest_pt_world[0]
    contact_world_frame_corrected.pose.position.y = nearest_pt_world[1]
    contact_world_frame_corrected.pose.position.z = nearest_pt_world[2]

    contact_obj_frame_corrected = util.pose_stamped2list(util.convert_reference_frame(contact_world_frame_corrected, util.list2pose_stamped(start), util.unit_pose()))
    new_data = copy.deepcopy(data)
    new_data['contact_obj_frame'] = contact_obj_frame_corrected
    return new_data

In [ ]:
# scene = vis_palms(grasp_data, name='grasp', goal='goal')
scene = vis_palms(get_rand_data('grasp'), name='grasp', goal='transformation')
# scene = vis_palms(pull_data, name='pull', goal='transformation')
# scene = vis_palms(get_rand_data('pull'), name='pull', goal='transformation')

scene.show(viewer='gl')

In [ ]:
z_sample = torch.randn_like(torch.zeros(3, ))

x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))
decoder_input = x[:31]

pose = vae.decode(z_sample, decoder_input)
pose_r = pose[0]
pose_l = pose[1]

pull_data_eval = copy.deepcopy(pull_data)
pull_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
pull_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

scene = vis_palms(pull_data_eval, name='pull')

# grasp_data_eval = copy.deepcopy(grasp_data)
# grasp_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# grasp_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

# scene = vis_palms(grasp_data_eval, name='grasp')
scene.show(viewer='gl')

In [ ]:
# scene = vis_palms(pull_data_eval, name='pull')
scene = vis_palms(grasp_data_eval, name='grasp')

scene.show(viewer='gl')

In [ ]:
# scene = vis_palms(correct_pos(pull_data_eval), name='pull')
# scene = vis_palms(grasp_data, name='grasp')

# scene.show(viewer='gl')

In [ ]:
# x = torch.from_numpy(np.asarray(start+goal, dtype=np.float32))
# output = vae.forward(x)

# pos = output[1][:3].data.cpu().numpy()
# ori = output[1][3:].data.cpu().numpy()

# ori = ori/np.linalg.norm(ori)

# pull_data_eval = copy.deepcopy(pull_data)
# pull_data_eval['contact_obj_frame'] = pos.tolist() + ori.tolist()

# scene = vis_palms(pull_data_eval, name='pull')

# x = torch.from_numpy(np.hstack((keypoints_start.flatten().astype(np.float32), np.asarray(goal, dtype=np.float32))))
# output = vae.forward(x)

# pose_r = output[1][:7]
# pose_l = output[1][7:]

In [ ]:
outputs = []
z = torch.randn_like(torch.zeros(500, 2))
decoder_inputs = decoder_input.repeat(500).reshape(500, decoder_input.shape[0])

output = vae.decode(z, decoder_inputs)

In [ ]:
# output_mean = []
# output_std = []
# for i in range(output[0].shape[1]):
#     avg_dim_i = torch.mean(output[0][:, i], axis=0)
#     std_dim_i = torch.std(output[0][:, i], axis=0)
#     print(avg_dim_i)
#     output_mean.append(avg_dim_i.data.cpu())
#     output_std.append(std_dim_i.data.cpu())

output_mean = []
output_std = []
for i in range(output[0].shape[1]):
    avg_dim_i_r = torch.mean(output[0][:, i], axis=0)
    std_dim_i_r = torch.std(output[0][:, i], axis=0)
    
    avg_dim_i_l = torch.mean(output[1][:, i], axis=0)
    std_dim_i_l = torch.std(output[1][:, i], axis=0)    
    print(avg_dim_i)
    output_mean.append((avg_dim_i_r.data.cpu(), avg_dim_i_l.data.cpu()))
    output_std.append((std_dim_i_r.data.cpu(), avg_dim_i_l.data.cpu()))

In [ ]:
# pos = np.asarray(output_mean[:3])
# ori = np.asarray(output_mean[3:])

# ori = ori/np.linalg.norm(ori)

# pull_data_eval = copy.deepcopy(pull_data)
# pull_data_eval['contact_obj_frame'] = pos.tolist() + ori.tolist()

# scene = vis_palms(pull_data_eval, name='pull')

# pose_r = output_mean[:7]
# pose_l = output_mean[7:]

# pose_r = output_mean[0][:7]
# pose_l = output_mean[1][7:]

pose_r, pose_l = [], []
for i in range(output[0].shape[1]):
    pose_r.append(output_mean[i][0])
    pose_l.append(output_mean[i][1])

grasp_data_eval = copy.deepcopy(grasp_data)
grasp_data_eval['contact_obj_frame']['right'] = pose_r
grasp_data_eval['contact_obj_frame']['left'] = pose_l
scene = vis_palms(grasp_data_eval, name='grasp')

scene.show(viewer='gl')

In [ ]:
loss_data = np.load('/root/training/saved_models/grasp_keypoints_two_heads_full_data_grasp_input_1/grasp_keypoints_two_heads_full_data_grasp_input_1_epoch_199_recon_loss.npz', allow_pickle=True)
loss_data.keys()
loss_data['pos_loss'].shape

In [ ]:
plt.figure()
plt.plot(loss_data['pos_loss'])
plt.show()
plt.title('Position loss')
plt.ylabel('Loss')

In [ ]:
plt.figure()
plt.plot(loss_data['ori_loss'])
plt.show()
plt.title('Orientation loss')
plt.ylabel('Loss')

In [ ]:
# sos = signal.butter(3, 0.03, 'low', output='sos')
sos = signal.butter(3, 0.2, 'low', output='sos')
filtered = signal.sosfilt(sos, loss_data['ori_loss'])

plt.figure()
plt.plot(filtered[4:])
plt.show()

# Testing setting goals

In [ ]:
x = torch.from_numpy(keypoints_start.flatten().astype(np.float32))
x.shape

In [ ]:
# contact_input = torch.from_numpy(np.asarray(contact_r+contact_l, dtype=np.float32))
# contact_input = torch.from_numpy(np.asarray(contact_r, dtype=np.float32))

decoder_input = goal
z, pose_sample, z_mu, z_logvar = output = vae.forward(x, decoder_input)

In [ ]:
z_sample = torch.randn_like(torch.zeros(2, ))

x = torch.from_numpy(keypoints_start.flatten().astype(np.float32))

pose = vae.decode(z_sample, x)
pose_object = pose

# pull_data_eval = copy.deepcopy(pull_data)
pull_data_eval = copy.deepcopy(pull_data)
# pull_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# pull_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)
pull_data_eval['goal'] = pose_to_list(pose_object)

scene = vis_palms(pull_data_eval, name='pull')

# grasp_data_eval = copy.deepcopy(grasp_data)
# grasp_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# grasp_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

# scene = vis_palms(grasp_data_eval, name='grasp')
scene.show(viewer='gl')

In [ ]:
# grasp_data_eval = get_rand_data('grasp')
# scene = vis_palms(grasp_data_eval, name='grasp', goal='transformation')

pull_data_eval = get_rand_data('pull')
scene = vis_palms(pull_data_eval, name='pull', goal='transformation')

scene.show(viewer='gl')

# Test transformations

In [ ]:
z_sample = torch.randn_like(torch.zeros(4, ))

# x = torch.from_numpy(keypoints_start.flatten().astype(np.float32))

trans = vae.decode(z_sample)

# # pull_data_eval = copy.deepcopy(pull_data)
# # pull_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# # pull_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)
# pull_data_eval = copy.deepcopy(pull_data)

pull_data_eval['transformation_corrected'] = pose_to_list(trans)
scene = vis_palms(pull_data_eval, name='pull', goal='transformation')

# grasp_data_eval = copy.deepcopy(grasp_data)
# grasp_data_eval['contact_obj_frame']['right'] = pose_to_list(pose_r)
# grasp_data_eval['contact_obj_frame']['left'] = pose_to_list(pose_l)

# grasp_data_eval['transformation_corrected'] = pose_to_list(trans)
# scene = vis_palms(grasp_data_eval, name='grasp', goal='transformation')

scene.show(viewer='gl')

In [ ]:
total_loss = np.load('/root/training/saved_models/grasp_transformation_small_batch_lr_1e2_ratio_test_0/grasp_transformation_small_batch_lr_1e2_ratio_test_0_epoch_199_loss.npz', allow_pickle=True)

In [ ]:
total_loss['loss'][-1].keys()

In [ ]:
kl = []
pos = []
ori = []
recon = []
for i, val in enumerate(total_loss['loss']):
    kl.append(val['epoch_kl'])
    pos.append(val['epoch_pos'])
    ori.append(val['epoch_ori'])
    recon.append(val['epoch_recon'])

In [ ]:
%matplotlib notebook

In [ ]:
plt.subplot(211)
plt.plot(np.asarray(kl))
plt.legend(['kl'])
plt.subplot(212)
plt.plot(np.asarray(pos))
plt.plot(np.asarray(ori))
plt.legend(['pos', 'ori'])
plt.show()

In [ ]:
print(np.argmin(pos))
print(np.argmin(ori))
print(np.argmin(recon))
print(np.argmin(kl))